# Round 3 verification 

In [2]:
import os
import sys
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/')) # Assuming the DummyDataGenerator is defined in a module

import numpy as np
import pandas as pd
from  round3_op_implementation import ProjectAllocator
from model.DummyDataGenerator import DummyDataGenerator 
from model.VotingRules import VotingRules

# Constants for allocation
QUORUM = 17
MIN_AMOUNT = 1500
TOTAL_AMOUNT = 30000000

# Generate dummy data

generator = DummyDataGenerator(
    num_rows=100,  # Number of rows (voters) to generate
    max_project_in_ballot=5,  # Maximum projects a voter can vote for
    max_votes=3,  # Maximum votes a voter can cast
    start_date="2023-01-01",  # Start date for random timestamps
    end_date="2023-12-31"  # End date for random timestamps
)

# Generate the DataFrame
df = generator.generate_dummy_data()

# Expand the 'Votes' column and format the DataFrame for ProjectAllocator
df_votes_expanded = df.explode('Votes').reset_index(drop=True)
df_votes_expanded = pd.concat(
    [df_votes_expanded.drop(['Votes'], axis=1), df_votes_expanded['Votes'].apply(pd.Series)],
    axis=1
)
df_votes_expanded = df_votes_expanded.rename(columns={
    "projectId": "project_id",
    "amount": "amount",
    "Address": "voter_address"
})
df_votes_expanded["amount"] = df_votes_expanded["amount"].astype(float)  # Ensure amount is a float

# Instantiate the allocator and calculate initial and scaled allocations
allocator = ProjectAllocator(TOTAL_AMOUNT, MIN_AMOUNT, QUORUM)
initial_allocation = allocator.calculate_initial_allocation(df_votes_expanded)
print('Initial_Allocation')
print(initial_allocation)

scaled_allocation = allocator.scale_allocations(initial_allocation, "median_amount")
print('Scaled_Allocation')
print(scaled_allocation)

# Now let's use your VotingRules class to calculate the allocation using the median rule
voting_matrix = df_votes_expanded.pivot_table(
    index='voter_address', columns='project_id', values='amount', fill_value=0
).values


num_voters = voting_matrix.shape[0]
total_op_tokens = TOTAL_AMOUNT  # Equivalent to total funds in this context


voting_rules = VotingRules()
median_allocation = voting_rules.r3_median(voting_matrix, total_op_tokens, num_voters)

comparison_df = pd.DataFrame({
    'Allocation (ProjectAllocator)': scaled_allocation['scaled_amount'].values,
    'Allocation (Our Implemenation)': median_allocation
})
print('Comparison_Allocation')
comparison_df.head()

2024-08-14 08:32:40 INFO | Check - If all are eligible: True
2024-08-14 08:32:40 INFO | Check - Original Amount Eligible: 11084481.0
2024-08-14 08:32:40 INFO | Check - Scale Factor: 2.7064866636516407
2024-08-14 08:32:40 INFO | Check - New Amount Eligible: 30000000.000000004


Initial_Allocation
            votes_count  median_amount  is_eligible
project_id                                         
proj0                18      1533100.5         True
proj1                28      2787411.5         True
proj2                24      2329909.0         True
proj3                24      2170789.0         True
proj4                21      2263271.0         True
Scaled_Allocation
            votes_count  median_amount  is_eligible  scaled_amount
project_id                                                        
proj0                18      1533100.5         True   4.149316e+06
proj1                28      2787411.5         True   7.544092e+06
proj2                24      2329909.0         True   6.305868e+06
proj3                24      2170789.0         True   5.875211e+06
proj4                21      2263271.0         True   6.125513e+06
Comparison_Allocation


,Allocation (ProjectAllocator),Allocation (Our Implemenation)
0,4.149316e+06,4.149316e+06
1,7.544092e+06,7.544092e+06
2,6.305868e+06,6.305868e+06
3,5.875211e+06,5.875211e+06
4,6.125513e+06,6.125513e+06


# Round 2 Verification

In [4]:
import numpy as np

# Define the voting matrix
voting_matrix = np.array([[50, 10, 40, 0],
                          [10, 50, 20, 20],
                          [0, 50, 0, 50],
                          [0, 100, 0, 0],
                          [40, 20, 20, 20]])
total_op_tokens=10e6
num_voters=5

voting_rules = VotingRules()
mean_allocation = voting_rules.r2_mean(voting_matrix, total_op_tokens, num_voters)
print(mean_allocation)

[2000000. 4600000. 1600000. 1800000.]
